In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h1>Ипорт библиотек</h1>

In [ ]:
from sklearn.metrics import accuracy_score #Import accuracy model library.

In [ ]:
df = pd.read_csv('../input/bnp-paribas-cardif-claims-management/train.csv.zip')
df.shape

In [ ]:
df.head()

# Data preprocessing 

In [ ]:
df = df.drop("ID", axis=1)

In [ ]:
df.isnull().sum()

## Сплит на трейн, валидацию и тест 

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [ ]:
train.shape

## Заполнение nan

In [ ]:
train = train.fillna(train.median())
validate = validate.fillna(validate.median())

## Выделение уникальных данных 

In [ ]:
from sklearn import preprocessing

for f in train.columns:
    if train[f].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(np.unique(list(train[f].values)  + list(validate[f].values)))
        train[f]   = lbl.transform(list(train[f].values))
        validate[f]  = lbl.transform(list(validate[f].values))


## Индексация данных

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(train.select_dtypes(include=numerics).columns)
train = train[numerical_vars]
numerical_vars2 = list(validate.select_dtypes(include=numerics).columns)
validate = validate[numerical_vars]

## Выделение таргета

In [ ]:
X_train = train.iloc[:,train.columns != 'target']
y_train = train.iloc[:, train.columns == 'target']
X_validate = validate.iloc[:,validate.columns != 'target']
y_validate = validate.iloc[:, validate.columns == 'target']

## Feature Selection

In [ ]:
from sklearn.feature_selection import f_classif, SelectKBest

univariate = f_classif(X_train, y_train)

univariate = pd.Series(univariate[1])
univariate.index = X_train.columns

## Data visualization 

In [ ]:
univariate.sort_values(ascending=False).plot.bar(figsize=(20, 8))

## Выбор топ 20 предикторов

In [ ]:
sel_ = SelectKBest(f_classif, k=20).fit(X_train, y_train)
X_train.columns[sel_.get_support()]

In [ ]:
columns = ['v4', 'v10', 'v12', 'v14', 'v21', 'v31', 'v34', 'v38', 'v47', 'v50',
       'v56', 'v62', 'v66', 'v72', 'v110', 'v113', 'v114', 'v119', 'v123',
       'v129']
X_train = X_train[columns]
X_validate = X_validate[columns]

# Modelling

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_validate)
accuracy_score(y_validate, y_pred)

# GINI

In [ ]:
def G(x):
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

G(y_pred)